In [1]:
import json
import pandas as pd
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from pyspark.sql.functions import col, count, desc,concat_ws
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType, StringType
from langdetect import detect


In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 19:08:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/14 19:08:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


23/12/14 19:08:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
# Read the dataset in Spark
df = spark.read.json('/Users/celine/Desktop/5430nlp/group/deduplicated_reviews.json')

In [5]:
df.show(5)

+--------------+--------------------+--------------------+--------------------+------------+----+---------------------+----------+-----+--------------------+-------+--------+--------+-----------+-------------+-----------+------------+---------+-----+-----+--------------------+------+--------------------+
|       address|          attributes|         business_id|          categories|        city|cool|customer_review_stars|      date|funny|               hours|is_open|language|latitude|  longitude|         name|postal_code|review_count|review_id|stars|state|                text|useful|             user_id|
+--------------+--------------------+--------------------+--------------------+------------+----+---------------------+----------+-----+--------------------+-------+--------+--------+-----------+-------------+-----------+------------+---------+-----+-----+--------------------+------+--------------------+
|2654 S 10th St|Row(AcceptsInsura...|RI33oswGDkIsc0fuQ...|Restaurants, Chee...|Phi

In [8]:
# select related columns
df = df.select('business_id', 'name', 'stars', 'address', 'city', 'postal_code', 'text')
df.show(5)

+--------------------+-------------+-----+--------------+------------+-----------+--------------------+
|         business_id|         name|stars|       address|        city|postal_code|                text|
+--------------------+-------------+-----+--------------+------------+-----------+--------------------+
|RI33oswGDkIsc0fuQ...|Oregon Steaks|  3.5|2654 S 10th St|Philadelphia|      19148|after several ppl...|
|RI33oswGDkIsc0fuQ...|Oregon Steaks|  3.5|2654 S 10th St|Philadelphia|      19148|The cheeseburgers...|
|RI33oswGDkIsc0fuQ...|Oregon Steaks|  3.5|2654 S 10th St|Philadelphia|      19148|In my humble opin...|
|RI33oswGDkIsc0fuQ...|Oregon Steaks|  3.5|2654 S 10th St|Philadelphia|      19148|Overall this plac...|
|RI33oswGDkIsc0fuQ...|Oregon Steaks|  3.5|2654 S 10th St|Philadelphia|      19148|Perfect place for...|
+--------------------+-------------+-----+--------------+------------+-----------+--------------------+
only showing top 5 rows



In [9]:
# combine reviews together for each business
combined_reviews_df = df.groupBy('business_id').agg(concat_ws(" ", collect_list(col('text'))).alias('combined_reviews'))
combined_reviews_df,show(5)

NameError: name 'concat_ws' is not defined

# Data cleaning

In [7]:
columns_to_keep = ['business_id', 'name', 'stars', 'address', 'city', 'postal_code', 'text']
df = df[columns_to_keep]

#combine all the reviews together
combined_text = df.groupby('business_id')['text'].apply(' '.join).reset_index()

# Merging the aggregated 'text' with the original data based on 'name'
df = pd.merge(df.drop('text', axis=1).drop_duplicates(subset=['business_id']), combined_text, on='business_id')

# Printing the resulting DataFrame
print(df)

                 business_id                             name  stars  \
0     RI33oswGDkIsc0fuQ8GzjA                    Oregon Steaks    3.5   
1     MTSW4McQd7CbVtyjqoe9mw               St Honore Pastries    4.0   
2     GmJDCmDhtKbjofeO35yBrw  The Freshworks of Port Richmond    4.5   
3     y1HgVnt4K5owmYXC5yXLGg      Flambo Caribbean Restaurant    4.5   
4     DOl21AqtT74dUZ3kvXNNcA                  Golden Triangle    4.5   
...                      ...                              ...    ...   
2271  Mj0eEQNzeL3kSPwSH8yRVA                          Buccann    3.5   
2272  8OoLxv91ztELwzfvG26xjA   Taste Queen Chinese Restaurant    3.5   
2273  NyH11DJUjiGIrSq7n8IDSQ                     ACME Markets    2.5   
2274  6CpCnFzpyOV4ymhNP1AOGw               Tamalex Restaurant    4.5   
2275  NQn2fIomrrMXXUxZXH99YA                         Rox Cafe    2.5   

                address          city  postal_code  \
0        2654 S 10th St  Philadelphia        19148   
1           935 Race St  Ph

In [8]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer


# Step 1: Convert 'text' column to lowercase
df['text'] = df['text'].str.lower()

# Step 2: Remove specified words from 'text' column
words_to_remove = ['bite', 'bit', 'restaurant', 'business', 'dish', 'meal', 'wait', 'waiter', 'waitress', 'favorite',
                   'person', 'manager', 'size', 'point', 'husband', 'dine', 'course', 'home', 'ice', 'item', 'line',
                   'location', 'owner', 'portion', 'selection', 'server', 'service', 'food', 'street', 'table',
                   'taste', 'friend', 'plate', 'bowl', 'water', 'week', 'weekend', 'space', 'room', 'friend']
for word in words_to_remove:
    df['text'] = df['text'].str.replace(r'\b' + word + r'\b', '', regex=True)

# Step 3: Tokenize and lemmatize, keeping only nouns
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    nouns = [token for token, pos in nltk.pos_tag(lemmatized_tokens) if pos.startswith('N')]
    return ' '.join(nouns)

df['text'] = df['text'].apply(lemmatize_text)

# Step 4: Create clusters
def get_cluster(text):
    if 'japanese' in text:
        return 'japanese'
    elif 'pizza' in text:
        return 'pizza'
    elif 'burger' in text:
        return 'burger'
    elif 'korean' in text:
        return 'korean'
    elif 'italian' in text:
        return 'italian'
    elif 'american' in text:
        return 'american'
    elif 'chinese' in text:
        return 'chinese'
    elif 'vegan' in text:
        return 'vegan'
    elif 'thai' in text:
        return 'thai'
    else:
        return 'others'

df['cluster'] = df['text'].apply(get_cluster)

[nltk_data] Downloading package punkt to /Users/shining/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shining/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/shining/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
print(df)

                 business_id                             name  stars  \
0     RI33oswGDkIsc0fuQ8GzjA                    Oregon Steaks    3.5   
1     MTSW4McQd7CbVtyjqoe9mw               St Honore Pastries    4.0   
2     GmJDCmDhtKbjofeO35yBrw  The Freshworks of Port Richmond    4.5   
3     y1HgVnt4K5owmYXC5yXLGg      Flambo Caribbean Restaurant    4.5   
4     DOl21AqtT74dUZ3kvXNNcA                  Golden Triangle    4.5   
...                      ...                              ...    ...   
2271  Mj0eEQNzeL3kSPwSH8yRVA                          Buccann    3.5   
2272  8OoLxv91ztELwzfvG26xjA   Taste Queen Chinese Restaurant    3.5   
2273  NyH11DJUjiGIrSq7n8IDSQ                     ACME Markets    2.5   
2274  6CpCnFzpyOV4ymhNP1AOGw               Tamalex Restaurant    4.5   
2275  NQn2fIomrrMXXUxZXH99YA                         Rox Cafe    2.5   

                address          city  postal_code  \
0        2654 S 10th St  Philadelphia        19148   
1           935 Race St  Ph

In [10]:
cluster_counts = df['cluster'].value_counts()
print(cluster_counts)

cluster
others      782
pizza       615
burger      437
vegan       136
thai         81
american     67
chinese      63
italian      50
japanese     23
korean       22
Name: count, dtype: int64


In [11]:
df.to_csv('processed_data.csv')

# Topic modelling

In [ ]:
#!pip install -U sklearn
#!pip install -U pyLDAvis

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

In [18]:
param_grid = {
    'n_components': [5, 10, 15],  # Try different numbers of topics
    'learning_method': ['batch', 'online'],
    'max_iter': [100, 500, 1000] 
    # Add other parameters to the grid
}

# Create CountVectorizer instance
tf_vectorizer = CountVectorizer(strip_accents='unicode', 
                                stop_words='english',
                                lowercase=True,
                                max_features=100,
                                token_pattern=r'\b[a-zA-Z]{3,}\b',
                                max_df=0.7,
                                min_df=100,
                                ngram_range=(1, 3))

# Transform the text data
tf = tf_vectorizer.fit_transform(df['text'])

In [19]:
from gensim.models import CoherenceModel

# Create LDA model
lda = LatentDirichletAllocation()

# GridSearchCV for parameter tuning
grid_search = GridSearchCV(lda, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(tf)  

# Get the results as a DataFrame
results = pd.DataFrame(grid_search.cv_results_)

# Calculate perplexity for each parameter combination
perplexities = []
for params in grid_search.cv_results_['params']:
    lda_model = LatentDirichletAllocation(**params)
    lda_model.fit(tf)
    perplexity = lda_model.perplexity(tf)
    perplexities.append(perplexity)

# Add perplexity scores to the results DataFrame
results['perplexity'] = perplexities

# Display the table with perplexity scores for each parameter combination
print(results[['param_n_components', 'param_learning_method', 'param_max_iter', 'perplexity']])

# Best parameters
print(grid_search.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END learning_method=batch, max_iter=100, n_components=5; total time=  30.1s
[CV] END learning_method=batch, max_iter=100, n_components=15; total time=  41.5s
[CV] END learning_method=batch, max_iter=500, n_components=5; total time= 2.9min
[CV] END learning_method=batch, max_iter=500, n_components=15; total time= 3.1min
[CV] END learning_method=batch, max_iter=1000, n_components=10; total time= 5.8min
[CV] END learning_method=batch, max_iter=1000, n_components=15; total time= 6.1min
[CV] END learning_method=online, max_iter=500, n_components=10; total time= 3.1min
[CV] END learning_method=online, max_iter=500, n_components=15; total time= 3.5min
[CV] END learning_method=online, max_iter=1000, n_components=10; total time= 5.9min
[CV] END learning_method=batch, max_iter=100, n_components=10; total time=  34.2s
[CV] END learning_method=batch, max_iter=100, n_components=15; total time=  38.0s
[CV] END learning_method=batch, m

In [20]:
#best LDA model
lda_tf = LatentDirichletAllocation(n_components=15, 
                                   max_iter=1000,
                                   learning_method='online', 
                                   random_state = 28474460)
lda_tf.fit(tf)

LatentDirichletAllocation(learning_method='online', max_iter=1000,
                          n_components=15, random_state=28474460)

In [74]:
#Examine the LDA features
tf_feature_names = tf_vectorizer.get_feature_names_out()

print('Number of features is:', len(tf_feature_names))
print('First 10 features:', tf_feature_names[:10])

Number of features is: 100
First 10 features: ['amaze' 'ambiance' 'appetizer' 'bagel' 'bar' 'bartender' 'beef' 'beer'
 'bread' 'breakfast']


In [32]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
pyLDAvis.lda_model.prepare(lda_tf, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.041730 -0.091384       1        1  19.447498
7     -0.114979 -0.182133       2        1  12.063811
10     0.142342 -0.042475       3        1   8.135744
6     -0.004076  0.225087       4        1   7.946283
8     -0.086614  0.035354       5        1   7.762937
9      0.140631 -0.050297       6        1   6.056317
14    -0.011769  0.089460       7        1   5.477813
0      0.065811  0.222637       8        1   5.454614
3     -0.200252 -0.141311       9        1   5.267752
11    -0.078190  0.119406      10        1   4.970539
13    -0.261045  0.052609      11        1   4.616605
2      0.147299 -0.097415      12        1   4.003973
1      0.038256 -0.042499      13        1   3.578779
12     0.169797 -0.017750      14        1   3.362355
4      0.094519 -0.079291      15        1   1.854981, topic_info=      Term          Freq         Total Category  logprob  loglift
60   pizza  15639.000000  15639.000000  Default  30.0000  30.0000
68    roll   9869.000000   9869.000000  Default  29.0000  29.0000
87    taco   6959.000000   6959.000000  Default  28.0000  28.0000
41     fry  14666.000000  14666.000000  Default  27.0000  27.0000
21  coffee   8759.000000   8759.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
35     eat     50.672951   4704.129668  Topic15  -5.3424  -0.5435
52   night     81.180842  12348.927457  Topic15  -4.8711  -1.0373
94   visit     56.697256   6487.089674  Topic15  -5.2301  -0.7525
4      bar     65.380627  13127.661908  Topic15  -5.0876  -1.3150
40  friend     53.856348   6982.591828  Topic15  -5.2815  -0.8776

[545 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.341894  amaze
0         2  0.061221  amaze
0         3  0.102636  amaze
0         4  0.054019  amaze
0         5  0.090706  amaze
...     ...       ...    ...
98       14  0.002915   wine
99        2  0.212270   wing
99        3  0.027582   wing
99        7  0.522428   wing
99        8  0.237689   wing

[1053 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 11, 7, 9, 10, 15, 1, 4, 12, 14, 3, 2, 13, 5])

In [ ]:
n_top_words = 3  # Number of top words to extract

# Extract top words for each review
reviews = {}
for idx, review in enumerate(tf):  # Assuming 'tf' is your topic-token matrix
    topic = lda_tf.transform(review.reshape(1, -1))  # Transform the review to match LDA input
    ordered_indexes = topic.argsort()[0][::-1]
    top_indexes = ordered_indexes[:n_top_words]
    top_features = [tf_feature_names[i] for i in top_indexes]
    reviews[idx] = top_features

# Add a new 'topic' column to df1 containing the top 3 words for each review
df['topic'] = [reviews[i] for i in range(len(df))]

In [71]:
print(df)

                 business_id                             name  stars  \
0     RI33oswGDkIsc0fuQ8GzjA                    Oregon Steaks    3.5   
1     MTSW4McQd7CbVtyjqoe9mw               St Honore Pastries    4.0   
2     GmJDCmDhtKbjofeO35yBrw  The Freshworks of Port Richmond    4.5   
3     y1HgVnt4K5owmYXC5yXLGg      Flambo Caribbean Restaurant    4.5   
4     DOl21AqtT74dUZ3kvXNNcA                  Golden Triangle    4.5   
...                      ...                              ...    ...   
2271  Mj0eEQNzeL3kSPwSH8yRVA                          Buccann    3.5   
2272  8OoLxv91ztELwzfvG26xjA   Taste Queen Chinese Restaurant    3.5   
2273  NyH11DJUjiGIrSq7n8IDSQ                     ACME Markets    2.5   
2274  6CpCnFzpyOV4ymhNP1AOGw               Tamalex Restaurant    4.5   
2275  NQn2fIomrrMXXUxZXH99YA                         Rox Cafe    2.5   

                address          city  postal_code  \
0        2654 S 10th St  Philadelphia        19148   
1           935 Race St  Ph

In [80]:
# Convert the first character of each word in the 'topic' column to uppercase
df['topic'] = df['topic'].apply(lambda x: [word.capitalize() for word in x])
print(df)

                 business_id                             name  stars  \
0     RI33oswGDkIsc0fuQ8GzjA                    Oregon Steaks    3.5   
1     MTSW4McQd7CbVtyjqoe9mw               St Honore Pastries    4.0   
2     GmJDCmDhtKbjofeO35yBrw  The Freshworks of Port Richmond    4.5   
3     y1HgVnt4K5owmYXC5yXLGg      Flambo Caribbean Restaurant    4.5   
4     DOl21AqtT74dUZ3kvXNNcA                  Golden Triangle    4.5   
...                      ...                              ...    ...   
2271  Mj0eEQNzeL3kSPwSH8yRVA                          Buccann    3.5   
2272  8OoLxv91ztELwzfvG26xjA   Taste Queen Chinese Restaurant    3.5   
2273  NyH11DJUjiGIrSq7n8IDSQ                     ACME Markets    2.5   
2274  6CpCnFzpyOV4ymhNP1AOGw               Tamalex Restaurant    4.5   
2275  NQn2fIomrrMXXUxZXH99YA                         Rox Cafe    2.5   

                address          city  postal_code  \
0        2654 S 10th St  Philadelphia        19148   
1           935 Race St  Ph

In [51]:
df.to_json('tm.json', orient='records')